# Convert to ONNX


In [1]:
# !pip install onnxruntime onnx transformers optimum

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import torch
from torch import optim
import torch.nn.functional as F

from transformers import BertForSequenceClassification, BertConfig, BertTokenizer

from pathlib import Path
import timeit
import onnxruntime as ort
from onnxruntime import InferenceSession
from optimum.onnxruntime import ORTModelForSequenceClassification

Converting a model to ONNX format is straightforward. We just have to instantiate the model with the `ORTModelForSequenceClassification` from_pretrained function.

In [5]:
# Create a PATH to save the model
model_onnx_path = Path("/content/drive/MyDrive/Models/indobert-onnx")

# Load the model converted to ORT (ONNX)
model_load_path = Path("/content/drive/MyDrive/Models/indobert")
model_onnx = ORTModelForSequenceClassification.from_pretrained(model_load_path, export=True)

# Save the model
model_onnx.save_pretrained(model_onnx_path)

Framework not specified. Using pt to export to ONNX.
Using framework PyTorch: 2.0.1+cu118
Overriding 1 configuration item(s)
	- use_cache -> False


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



The inputs format for ONNX is a bit different than the usual. We have to declare it in a dictionary.

In [6]:
tokenizer = BertTokenizer.from_pretrained(model_load_path)

# Create inputs for the model
text = ['Bahagia hatiku melihat pernikahan putri sulungku yang cantik jelita']
inputs = tokenizer(text)

inputs_onnx = dict(
    input_ids=np.array(inputs["input_ids"]).astype("int64"),
    attention_mask=np.array(inputs["attention_mask"]).astype("int64"),
    token_type_ids=np.array(inputs["token_type_ids"]).astype("int64")
)

inputs_onnx

{'input_ids': array([[    2,  4771, 10413,   722,  3300,  3466, 19227,   457,    34,
          2176, 17377,   155,     3]]),
 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 'token_type_ids': array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

Let's try to inference those inputs!

In [7]:
# Create ONNX session to do inference
sess = InferenceSession(str(model_onnx_path / "model.onnx"), providers=["CPUExecutionProvider"])

In [9]:
i2w = {0: 'positive', 1: 'neutral', 2: 'negative'}

# Do the inference
logits = sess.run(None, input_feed=inputs_onnx)[0]
label = torch.topk(torch.from_numpy(logits), k=1, dim=-1)[1].squeeze().item()
probability = F.softmax(torch.from_numpy(logits), dim=-1).squeeze()[label].item()

print(f"Label: {i2w[label]}\nProbability: {probability}")

Label: positive
Probability: 0.9998039603233337


It works!